In [1]:
import pandas as pd
import requests
from pandas_profiling import ProfileReport
import sweetviz as sv
import numpy as np

In [2]:
unnecessary_columns = ['V_ID','P_ID','C_SEV','C_YEAR','P_USER','P_PSN','V_YEAR','P_ISEV']

In [25]:
data = pd.read_csv('NCDB_1999_to_2014.csv',low_memory=False)

In [26]:
data.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,01,1,20,2,02,34,UU,1,5,...,01,06,1990,01,M,41,11,1,UU,1
1,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,01,M,19,11,1,UU,1
2,1999,01,1,20,2,02,34,UU,1,5,...,02,01,1987,02,F,20,13,2,02,2
3,1999,01,1,08,2,01,01,UU,5,3,...,01,01,1986,01,M,46,11,1,UU,1
4,1999,01,1,08,2,01,01,UU,5,3,...,99,NN,NNNN,01,M,05,99,2,UU,3


In [27]:
#Agregar columna con cantidad de personas por accidente
collision_columns = [col for col in data.columns if col[0]=="C"]
group_by_columns = collision_columns[:]
group_by_columns.append('P_ID')
data = data.merge(data[group_by_columns].groupby(group_by_columns).size().reset_index().drop('P_ID',axis=1).rename(columns={0:"A_PERS"}),on=collision_columns)

In [28]:
print(f'Numero de registros: {data.shape[0]}')
print(f'Numero de columnas: {data.shape[1]}')

Numero de registros: 13672837
Numero de columnas: 23


In [29]:
data.dtypes.to_dict()

{'C_YEAR': dtype('int64'),
 'C_MNTH': dtype('O'),
 'C_WDAY': dtype('O'),
 'C_HOUR': dtype('O'),
 'C_SEV': dtype('int64'),
 'C_VEHS': dtype('O'),
 'C_CONF': dtype('O'),
 'C_RCFG': dtype('O'),
 'C_WTHR': dtype('O'),
 'C_RSUR': dtype('O'),
 'C_RALN': dtype('O'),
 'C_TRAF': dtype('O'),
 'V_ID': dtype('O'),
 'V_TYPE': dtype('O'),
 'V_YEAR': dtype('O'),
 'P_ID': dtype('O'),
 'P_SEX': dtype('O'),
 'P_AGE': dtype('O'),
 'P_PSN': dtype('O'),
 'P_ISEV': dtype('O'),
 'P_SAFE': dtype('O'),
 'P_USER': dtype('O'),
 'A_PERS': dtype('int64')}

In [30]:
nas = ["UU","XX","U","X","XXXX","NN","NNNN","N","UUUU"]
others = {
    "P_SAFE" : "14",
    "P_PSN": "100",
    "V_TYPE": "24",
    "C_TRAF": "19",
    "C_RALN": "7",
    "C_RSUR": "10",
    "C_WTHR": "8",
    "C_RCFG": "13",
    "C_CONF": "42"
}
int_types = ["P_AGE","A_VAGE","A_PERS"]
categories = [col for col in data.columns if col not in int_types + unnecessary_columns]
target_columns = ["C_SEV","P_ISEV"]

In [31]:
#quedarse solo con los drivers
data = data[data['P_PSN']=="11"]

In [32]:
print("|".join(nas))

UU|XX|U|X|XXXX|NN|NNNN|N|UUUU


In [33]:
#data['C_MNTH'].replace(to_replace=rf'/^({"|".join(nas)})$/',value=pd.NA,regex=True)
#data['C_MNTH'].replace(to_replace=rf'^({"|".join(nas)})',value=pd.NA,regex=True)

In [34]:
#for na in nas:
#    data[categories] = data[categories].replace(na,pd.NA)

data[categories] = data[categories].replace(to_replace=rf'^({"|".join(nas)})',value=pd.NA,regex=True)

for key,value in others.items():
    data[key] = data[key].replace(to_replace=r'^Q.*', value=value, regex=True)
    
#data.replace(to_replace=r'^[A-Z].*',value=np.nan, regex=True,inplace=True)

In [35]:
categories

['C_MNTH',
 'C_WDAY',
 'C_HOUR',
 'C_VEHS',
 'C_CONF',
 'C_RCFG',
 'C_WTHR',
 'C_RSUR',
 'C_RALN',
 'C_TRAF',
 'V_TYPE',
 'P_SEX',
 'P_SAFE']

In [36]:
data['C_MNTH']

0             01
1             01
2             01
3             01
6             01
            ... 
13672827    <NA>
13672828    <NA>
13672829    <NA>
13672830    <NA>
13672832    <NA>
Name: C_MNTH, Length: 7620216, dtype: object

In [37]:
#agregar columna con antiguedad del coche
data['A_VAGE'] = data['C_YEAR'] - pd.to_numeric(data['V_YEAR'], errors='coerce')

In [38]:
#convertir las horas en 4 categorias
def transform_hour(value):
    value = pd.to_numeric(value,errors='coerce')
    if value >= 0 and value<6:
        return '0'
    if value >= 6 and value<12:
        return '1'
    if value >= 12 and value<18:
        return '2'
    if value >= 18 and value<24:
        return '3'
    return pd.NA

data['C_HOUR'] = data['C_HOUR'].apply(transform_hour) 
    

In [39]:
data[int_types] = data[int_types].apply(pd.to_numeric, errors='coerce',downcast='integer')
data[categories] = data[categories].astype('category')

In [40]:
data = data.rename(columns={'P_SEX':'A_DSEX','P_AGE':"A_DAGE",'C_HOUR':'A_CHUR'})

In [41]:
data = data.drop(unnecessary_columns,1)

In [42]:
data["C_MNTH"]

0            01
1            01
2            01
3            01
6            01
           ... 
13672827    NaN
13672828    NaN
13672829    NaN
13672830    NaN
13672832    NaN
Name: C_MNTH, Length: 7620216, dtype: category
Categories (12, object): ['01', '02', '03', '04', ..., '09', '10', '11', '12']

In [43]:
data.dtypes.to_dict()

{'C_MNTH': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                   '11', '12'],
                  ordered=False),
 'C_WDAY': CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7'], ordered=False),
 'A_CHUR': CategoricalDtype(categories=['0', '1', '2', '3'], ordered=False),
 'C_VEHS': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
                   '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
                   '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
                   '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
                   '41', '43', '44', '46', '51', '54', '56', '57', '58', '71',
                   '72', '77'],
                  ordered=False),
 'C_CONF': CategoricalDtype(categories=['01', '02', '03', '04', '05', '06', '21', '22', '23', '24',
                   '25', '31', '32', '33', '34', '35', '36', '41', '42'],
         

In [ ]:
#my_report = sv.analyze(data)
#my_report.show_notebook()
profile = ProfileReport(data, title="Crashes EDA")
profile.to_widgets()

In [ ]:
#data["C_MNTH"].unique()

In [ ]:
#data["P_ISEV"].isna().sum()

In [ ]:
#data["A_VAGE"].isna().sum()

In [ ]:
#data[data["A_PCOUNT"]==77]

In [18]:
print(categories)

['C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG', 'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_TYPE', 'P_SEX', 'P_SAFE']
